In [1]:
import pandas_datareader.data as web
import requests
import config

In [2]:
company = 'ROBLOX'
ticker = 'RBLX'
api = config.fmp_api_key

## Risk Free rate

In [3]:
start = 2017
end = 2022

periods_per_year = 1
# Load historical 5-year Treasury rate:
treasury_5yr_monthly = (web.DataReader('DGS5', 'fred', start, end)
                         .resample('M')
                         .last()
                         .div(periods_per_year)
                         .div(100)
                         .squeeze())

riskfree_rate = treasury_5yr_monthly.mean()
print("5-year historical average US Treasury rate is %.03f" % (riskfree_rate))

5-year historical average US Treasury rate is 0.016


## Company Default Risk

In [4]:
def interest_coverage(symbol):
  IS= requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}?apikey={api}').json()
  EBIT= IS[0]['ebitda'] - IS[0]['depreciationAndAmortization'] 
  interest_expense = IS[0]['interestExpense']
  interest_coverage_ratio = EBIT / interest_expense
  return interest_coverage_ratio

int_cov_ratio = interest_coverage(ticker); int_cov_ratio

-69.30165761646185

## Aswath Damodaran Synthetic rating
- Adapted from https://codingandfun.com/calculating-weighted-average-cost-of-capital-wacc-with-python/ 
- Using updated figures found here (https://people.stern.nyu.edu/adamodar/New_Home_Page/datafile/ratings.html)

In [5]:
def cost_of_debt(company, RF,interest_coverage_ratio):
  if interest_coverage_ratio > 12.5:
    #Rating is AAA
    credit_spread = 0.0067
  if (interest_coverage_ratio > 9.5) & (interest_coverage_ratio <= 12.499999):
    #Rating is AA
    credit_spread = 0.0082
  if (interest_coverage_ratio > 7.5) & (interest_coverage_ratio <=  9.499999):
    #Rating is A+
    credit_spread = 0.0103
  if (interest_coverage_ratio > 6) & (interest_coverage_ratio <=  7.499999):
    #Rating is A
    credit_spread = 0.0114
  if (interest_coverage_ratio > 4.5) & (interest_coverage_ratio <=  5.999999):
    #Rating is A-
    credit_spread = 0.0129
  if (interest_coverage_ratio > 4) & (interest_coverage_ratio <=  4.499999):
    #Rating is BBB
    credit_spread = 0.0159
  if (interest_coverage_ratio > 3.5) & (interest_coverage_ratio <=  3.999999):
    #Rating is BB+
    credit_spread = 0.0193
  if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  3.499999):
    #Rating is BB
    credit_spread = 0.0215
  if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  2.999999):
    #Rating is B+
    credit_spread = 0.0315
  if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.499999):
    #Rating is B
    credit_spread = 0.0378
  if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.999999):
    #Rating is B-
    credit_spread = 0.0462
  if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.499999):
    #Rating is CCC
    credit_spread = 0.0778
  if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.249999):
    #Rating is CC
    credit_spread = 0.0880
  if (interest_coverage_ratio > 0.5) & (interest_coverage_ratio <=  0.799999):
    #Rating is C
    credit_spread = 0.1076
  if interest_coverage_ratio <=  0.499999:
    #Rating is D
    credit_spread = 0.1434
  
  cost_of_debt = RF + credit_spread
  #print(cost_of_debt)
  return cost_of_debt

## Cost of debt

In [6]:
cost_debt = cost_of_debt(company,riskfree_rate,int_cov_ratio); cost_debt
print("Pre-tax cost of debt (borrowing) for %s is %.03f" % (company, cost_debt))

Pre-tax cost of debt (borrowing) for ROBLOX is 0.159
